In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 如何使用 Python 和 Matplotlib 创建漂亮的极坐标直方图


大家好，欢迎来到这个 Python + Matplotlib 教程，我将向您展示如何创建上面看到的漂亮的极坐标直方图。

当标准条形图(bar chart)的值太多时，极坐标直方图(Polar histograms)非常有用。每个条形向中间变细的圆形形状使我们能够将更多信息塞入同一区域。

我使用的是 [世界幸福报告](https://worldhappiness.report/)中的数据以及[世界银行](https://data.worldbank.org/)有关收入水平的信息。

[![Screenshot by the author](assets/r3ry0fy9yyuh7vknwgv7.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--KQKOg5Z3--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/r3ry0fy9yyuh7vknwgv7.png)

您可以在此 [GitHub 存储库](https://github.com/oscarleoo/polar-histogram-tutorial)中找到我正在使用的代码和数据。

让我们开始吧。

------

## 第1步：准备工作

让我们从一些准备工作开始。

### 导入库

我们只需要大家都熟悉的标准Python库。 PIL 不是强制性的，但它是我在添加标志时处理图像的首选。


In [ ]:
import math
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from matplotlib.lines import Line2D
from matplotlib.patches import Wedge
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


唯一突出的是最后一些特定的 Matplotlib 导入。我将在本教程后面介绍这些组件。

我使用 pandas 来加载数据。


In [ ]:
df = pd.read_csv("./hapiness_report_2022.csv", index_col=None)
df = df.sort_values("score").reset_index(drop=True)


### Seaborn 的风格设置

接下来，我使用 Seaborn 通过定义背景、文本颜色和字体来创建基本样式。


In [ ]:
font_family = "SimHei" #修改为中文字体 原值"PT Mono"
background_color = "#F8F1F1"
text_color = "#040303"

sns.set_style({
    "axes.facecolor": background_color,
    "figure.facecolor": background_color,
    "font.family": font_family,
    "text.color": text_color,
})


set_style 还有几个参数，但这四个是我在本教程中唯一需要的参数。

我使用  [Colorhunt](https://colorhunt.co/) 和 [Coolors](https://coolors.co/) 等网站来创建漂亮的调色板。

###  全局设置


我还添加了一些全局设置来控制总体外观。前四个定义直方图中楔形的范围、大小和宽度。


In [ ]:
START_ANGLE = 100 # At what angle to start drawing the first wedge
END_ANGLE = 450 # At what angle to finish drawing the last wedge
SIZE = (END_ANGLE - START_ANGLE) / len(df) # The size of each wedge
PAD = 0.2 * SIZE # The padding between wedges

INNER_PADDING = 2 * df.score.min()
LIMIT = (INNER_PADDING + df.score.max()) * 1.3 # Limit of the axes


内部填充(INNER_PADDING)在原点和每个楔形的起点之间创建距离。它在图表的中心位置开辟出一个空间，我可以在其中添加标题。

### 样板代码

作为一名软件工程师，我努力编写可重用的代码，当我从事数据可视化工作时也是如此。

这就是为什么我总是从创建几行样板代码开始，我可以使用可重用的函数来扩展它们。


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(30, 30))
ax.set(xlim=(-LIMIT, LIMIT), ylim=(-LIMIT, LIMIT))

for i, row in df.iterrows():
    bar_length = row.score
    name = row.country
    length = bar_length + INNER_PADDING
    start = 100 + i*SIZE + PAD
    end = 100 + (i+1)*SIZE
    angle = (end + start) / 2

    # Create variables here

    # Add wedge functions here

# Add general functions here

plt.axis("off")
plt.tight_layout()
plt.show()


在本教程的其余部分，我将在三个注释之一下创建并添加函数和变量。

------

## 第 2 步：绘制楔子

为了在 Matplotlib 中获得更多的视觉效果，使用底层组件而不是内置的图形函数会有所帮助。

### 绘制楔子

例如，您可以使用 `plt.patches.Wedge()` 绘制各个部分，而不是使用 `plt.pie()` 创建饼图。

这就是我创建以下函数的原因，该函数根据角度、长度、条形长度和颜色绘制楔形。


In [ ]:
def draw_wedge(ax, start_angle, end_angle, length, bar_length, color):
    ax.add_artist(
        Wedge((0, 0),
            length, start_angle, end_angle,
            color=color, width=bar_length
        )
    )


在样板代码中，我在“Add functions here在此处添加函数”注释下添加 `draw_wedge()` ，如下所示。


In [ ]:
bar_length = row.score
length = bar_length # + INNER_PADDING
start = 100 + i*SIZE + PAD
end = 100 + (i+1)*SIZE
.
.
.

# Add functions here在此处添加函数
draw_wedge(ax, start, end, length, bar_length, "#000")


我使用 `row.score` 定义 `bar_length` ，以便条形的可见部分彼此之间具有准确的尺寸关系。

现在，我删除了 `INNER_PADDING` 以向您展示它的作用。

当我运行代码时，我得到下图。

[![Initial chart with wedges](assets/lx8684cxbyv1kwgihcm2.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--BhnQ_rg_--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/lx8684cxbyv1kwgihcm2.png)

正如您所看到的，我们还有很长的路要走，直到我们得到类似于您一开始看到的极坐标直方图的东西，但至少我们已经成功地绘制了楔形。

我们在中间附近看到了很多视觉伪影，所以让我们取消注释 `INNER_PADDING` 。

这是我们得到的。

[![Chart with inner padding](assets/5bqusep04ntehpdv333h.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--BMvA7kEy--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/5bqusep04ntehpdv333h.png)

 好多了。

### 添加颜色

接下来，我有一个简单的颜色函数，可以根据该国家/地区的收入水平决定每个楔形的颜色。


In [ ]:
def color(income_group):
    if income_group == "High income":
        return "#468FA8"
    elif income_group == "Lower middle income":
        return "#E5625E"
    elif income_group == "Upper middle income":
        return "#62466B"
    elif income_group == "Low income":
        return "#6B0F1A"
    else:
        return "#909090"


我使用该函数作为 draw_wedge 函数的输入。


In [ ]:
# Add functions here    
draw_wedge(ax, start, end, length, bar_length, color(row.income))


这是结果。

[![Initial chart with wedges and color](assets/9w2d1t5u0j3010sr9xuo.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--L1Sn-c93--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/9w2d1t5u0j3010sr9xuo.png)

对于 `INNER_PADDING` 和 `color()` ，没有留下任何奇怪的痕迹。是时候添加信息来解释我们正在查看的内容了。

------

## 第 3 步：添加标签

让我们为极坐标直方图中的每个条形添加标签。我希望每个条形图都显示该国家的国旗、名称和幸福感得分。

### 确定位置

当您向 Matplotlib 中的图表添加国旗和文本时，您需要计算正确的位置。

这通常很棘手，特别是当你有一个不寻常的形状，就像我们在极坐标直方图中的那样。

下面的函数采用楔子的长度及其角度来计算位置。通过填充将位置推离（bar）一些，以增加一些视觉空间。


In [ ]:
def get_xy_with_padding(length, angle, padding):
    x = math.cos(math.radians(angle)) * (length + padding)
    y = math.sin(math.radians(angle)) * (length + padding)
    return x, y


我们可以将此函数用于国旗和文本。

### 添加国旗

对于国旗，我使用 FlatIcon 中的这些[圆形国旗](https://www.flaticon.com/packs/countrys-flags) 。

它们需要许可证，因此，不幸的是，我无法共享它们，但您可以在其他地方找到类似的标志。

这里我向图表添加标志的函数。它采用位置、国家/地区名称（对应于正确文件的名称）、缩放和旋转。


In [ ]:
def add_flag(ax, x, y, name, zoom, rotation):
    flag = Image.open("<location>/{}.png".format(name.lower()))
    flag = flag.rotate(rotation if rotation > 270 else rotation - 180)
    im = OffsetImage(flag, zoom=zoom, interpolation="lanczos", resample=True, visible=True)

    ax.add_artist(AnnotationBbox(
        im, (x, y), frameon=False,
        xycoords="data",
    ))


如果角度超过 270 度，我会更改旗帜的旋转方式。当我们开始在图表的右侧部分添加条形时，就会发生这种情况。此时，标志位于文本的左侧，更改旋转可以使阅读更加自然。

现在，我们可以计算角度，使用 `get_xy_with_padding()` 并在图表上放置国旗。


In [ ]:
bar_length = row.score
length = bar_length + INNER_PADDING
start = START_ANGLE + i*SIZE + PAD
end = START_ANGLE + (i+1)*SIZE

# Add variables here
angle = (end + start) / 2
flag_zoom = 0.004 * length
flag_x, flag_y = get_xy_with_padding(length, angle, 0.1 * length)

# Add functions here
...
add_flag(ax, flag_x, flag_y, row.country, flag_zoom, angle)


flag_zoom 参数决定标志的大小并取决于分数。如果一个国家得分较低，那么放置旗帜的空间就较小，我们需要将其缩小一点。

[![Polar histogram with flags](assets/cwwtemje68pu27d76uu6.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--a1b8rEI_--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/cwwtemje68pu27d76uu6.png)

极好的。

###添加国家/地区名称和分数

为了添加国家/地区的名称和分数，我编写了以下函数。

与标志一样，如果角度超过 270 度，我会更改旋转。否则，文字就会颠倒。


In [ ]:
def add_text(ax, x, y, country, score, angle):
    if angle < 270:
        text = "{} ({})".format(country, score)
        ax.text(x, y, text, fontsize=13, rotation=angle-180, ha="right", va="center", rotation_mode="anchor")
    else:
        text = "({}) {}".format(score, country)
        ax.text(x, y, text, fontsize=13, rotation=angle, ha="left", va="center", rotation_mode="anchor")


我们计算文本位置的方式与计算国旗的方式相同。

唯一的区别是我们添加了更多的填充，因为我们希望它距离楔子更远。


In [ ]:
bar_length = row.score
length = bar_length + INNER_PADDING
start = START_ANGLE + i*SIZE + PAD
end = START_ANGLE + (i+1)*SIZE

# Add variables here
angle = (end + start) / 2
flag_zoom = 0.004 * length
flag_x, flag_y = get_xy_with_padding(length, angle, 0.1 * length)
text_x, text_y = get_xy_with_padding(length, angle, 16*flag_zoom)

# Add functions here
...
add_flag(ax, flag_x, flag_y, row.country, flag_zoom, angle)
add_text(ax, text_x, text_y, row.country, bar_length, angle)


现在我们有了下图，它开始看起来好多了。

[![Polar histogram with flags and text](assets/gzzk776yoryit2dw1k8j.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--riNtfhR6--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/gzzk776yoryit2dw1k8j.png)

现在是时候告诉用户他们在看什么了。

------

## 第 4 步：添加信息


我们已经添加了所有数据。是时候通过添加有用的信息和指导来使图表可读了。

### 绘制参考线

参考线是一种优秀的视觉助手。它们在这里与标准条形图一样有效。

这个想法是在特定分数上画一条线，这间接帮助我们比较不同的国家。

这是我绘制参考线的函数。我重用 `draw_wedge()` 函数来绘制 0 到 360 度的楔形。


In [ ]:
def draw_reference_line(ax, point, size, padding, fontsize=18):
    draw_wedge(ax, 0, 360, point+padding+size/2, size, background_color)
    ax.text(-0.6, padding + point, point, va="center", rotation=1, fontsize=fontsize)


我为每个分数运行一次该函数以绘制多条参考线。


In [ ]:
# Add general functions here
draw_reference_line(ax, 2.0, 0.05, INNER_PADDING)
draw_reference_line(ax, 4.0, 0.05, INNER_PADDING)
draw_reference_line(ax, 6.0, 0.05, INNER_PADDING)


这是结果。

[![Polar histogram with reference lines](assets/hytwmkgc1yfqdwmta8lo.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--bT6yri64--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/hytwmkgc1yfqdwmta8lo.png)

这有很大的不同。

### 添加标题

图表中心间隙的目的是为标题创建一个自然的位置。将标题放在中间是不寻常的，可以立即吸引观众的兴趣。

添加标题的代码是标准 Matplotlib 功能。


In [ ]:
# Add general functions here
...
plt.title(
  "世界幸福指数报告2022".replace(" ", "\n"), 
  x=0.5, y=0.5, va="center", ha="center", 
  fontsize=64, linespacing=1.5
)


它看起来是这样的。

[![Polar histogram with title](assets/pflnil3zmwzbfk5q87qy.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--xrxORtG4--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/pflnil3zmwzbfk5q87qy.png)


就快要结束了，但我们还有一件事要做。

### 添加图例（legend）

观看者无法理解颜色的含义，但我们可以通过添加图例来解决这个问题。

为了添加图例，我创建了以下函数，该函数接受要添加的标签、它们的颜色和标题。


In [ ]:
def add_legend(labels, colors, title):
    lines = [
        Line2D([], [], marker='o', markersize=24, linewidth=0, color=c) 
        for c in colors
    ]

    plt.legend(
        lines, labels,
        fontsize=18, loc="upper left", alignment="left",
        borderpad=1.3, edgecolor="#E4C9C9", labelspacing=1,
        facecolor="#F1E4E4", framealpha=1, borderaxespad=1,
        title=title, title_fontsize=20,
    )


我将该函数添加到在此处添加常规函数“Add general functions here”下，并将其与其他所有内容一起运行。


In [ ]:
# Add general functions here
...

add_legend(
    labels=["High income", "Upper middle income", "Lower middle income", "Low income", "Unknown"],
    colors=["#468FA8", "#62466B", "#E5625E", "#6B0F1A", "#909090"],
    title="Income level according to the World Bank\n"
)


最终结果如下所示。

[![Polar histogram by the author](assets/2oe05tc9cyjh7x2nc79a.jpeg)](https://res.cloudinary.com/practicaldev/image/fetch/s--fv-412t8--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/2oe05tc9cyjh7x2nc79a.png)

就是这样。我们重新创建了您在顶部看到的美丽的极坐标直方图。

您的整个主代码块现在应该如下所示。


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(30, 30))
ax.set(xlim=(-LIMIT, LIMIT), ylim=(-LIMIT, LIMIT))

for i, row in df.iterrows():
    bar_length = row.score
    length = bar_length + INNER_PADDING
    start = START_ANGLE + i*SIZE + PAD
    end = START_ANGLE + (i+1)*SIZE
    angle = (end + start) / 2

    # Add variables here
    flag_zoom = 0.004 * length
    flag_x, flag_y = get_xy_with_padding(length, angle, 8*flag_zoom)
    text_x, text_y = get_xy_with_padding(length, angle, 16*flag_zoom)

    # Add functions here
    draw_wedge(ax, start, end, length, bar_length, color(row.income))
    add_flag(ax, flag_x, flag_y, row.country, flag_zoom, angle)
    add_text(ax, text_x, text_y, row.country, bar_length, angle)

ax.text(1-LIMIT, LIMIT-2, "+ main title", fontsize=58)

# Add general functions here
draw_reference_line(ax, 2.0, 0.06, INNER_PADDING)
draw_reference_line(ax, 4.0, 0.06, INNER_PADDING)
draw_reference_line(ax, 6.0, 0.06, INNER_PADDING)
plt.title("World Happiness Report 2022".replace(" ", "\n"), x=0.5, y=0.5, va="center", ha="center", fontsize=64, linespacing=1.5)

add_legend(
    labels=["High income", "Upper middle income", "Lower middle income", "Low income", "Unknown"],
    colors=["#468FA8", "#62466B", "#E5625E", "#6B0F1A", "#909090"],
    title="Income level according to the World Bank\n"
)

plt.axis("off")
plt.tight_layout()
plt.show()


这就是本教程的内容；恭喜你到达终点。

------

## 结论

今天，我们学习了使用 Matplotlib 和 Python 创建漂亮的极坐标直方图。

极坐标直方图的创建非常容易，使我们能够将更多信息填充到单个图表中。

我在本教程中使用了世界幸福报告，但您可以将其更改为另一个鼓舞人心的数据集。

我希望您学到一些技巧来帮助您将图表创意变为现实。
